In [7]:
!pip install --upgrade jupyter ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 2.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
!pip install --upgrade "sentence-transformers>=3.0.0"

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.7.0
    Uninstalling sentence-transformers-2.7.0:
      Successfully uninstalled sentence-transformers-2.7.0

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import requests 
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '04-monitoring/data/results-gpt4o-mini.csv'
docs_url = f'{base_url}/{relative_url}?raw=1'

df = pd.read_csv(docs_url)

# docs_response = requests.get(docs_url)
# documents = docs_response.json()

In [3]:
df = df.iloc[:300]

In [5]:
df.head(1)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp


In [11]:
model_name='multi-qa-mpnet-base-dot-v1'

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [12]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [13]:
encoded_ans = embedding_model.encode(answer_llm)

In [15]:
#Q1: What's the first value of the resulting vector?

encoded_ans[0]

-0.42244655

In [19]:
 df.iloc[0]

answer_llm     You can sign up for the course by visiting the...
answer_orig    Machine Learning Zoomcamp FAQ\nThe purpose of ...
document                                                0227b872
question                     Where can I sign up for the course?
course                                 machine-learning-zoomcamp
Name: 0, dtype: object

In [20]:
from tqdm import tqdm

evaluations=[]

# for elem in df.iterrows():
for _, elem in tqdm(df.iterrows(), total=len(df), desc="Processing answers"):
    answer_orig = elem['answer_orig'] 
    answer_llm = elem['answer_llm'] 
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    evaluations.append(v_llm.dot(v_orig))

Processing answers: 100%|██████████| 300/300 [01:23<00:00,  3.61it/s]


In [21]:
evaluations

[17.515987,
 13.418402,
 25.313255,
 12.147415,
 18.747736,
 33.970406,
 30.251705,
 29.521576,
 35.272198,
 27.751772,
 32.34471,
 31.441843,
 36.38072,
 33.340504,
 30.606163,
 32.503044,
 29.67445,
 24.35346,
 20.13247,
 23.99548,
 30.88028,
 32.692436,
 30.049173,
 16.078167,
 31.79642,
 37.980003,
 20.839046,
 32.612865,
 38.894196,
 34.051826,
 28.263878,
 27.124832,
 23.975264,
 26.340149,
 18.658115,
 25.016403,
 21.101131,
 33.72679,
 29.340347,
 28.654505,
 29.608585,
 30.810738,
 33.331203,
 26.220482,
 26.550072,
 13.148597,
 12.962547,
 12.275609,
 9.9744425,
 10.883928,
 29.845068,
 32.36178,
 22.18718,
 30.268936,
 25.091877,
 32.742783,
 28.220984,
 27.274975,
 24.208645,
 22.568907,
 19.767456,
 18.679333,
 20.422321,
 22.051325,
 18.188013,
 28.455883,
 25.919708,
 23.33234,
 22.20594,
 28.296299,
 39.23055,
 36.758507,
 31.913895,
 31.202858,
 36.913048,
 30.514185,
 36.26145,
 27.397552,
 37.792786,
 23.29768,
 34.252586,
 34.550625,
 30.316462,
 35.703526,
 31.0125

In [22]:
#Q2: What's the 75% cosine in the scores (non norm)?


import numpy as np


# Calculate the 75th percentile
percentile_75 = np.percentile(evaluations, 75)

print(f"The 75th percentile of the scores is: {percentile_75:.4f}")

The 75th percentile of the scores is: 31.6743


In [23]:
def normalize(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [24]:
evaluations_norm=[]

# for elem in df.iterrows():
for _, elem in tqdm(df.iterrows(), total=len(df), desc="Processing answers"):
    answer_orig = elem['answer_orig'] 
    answer_llm = elem['answer_llm'] 
    v_llm = embedding_model.encode(answer_llm)
    v_llm_norm = normalize(v_llm)
    v_orig = embedding_model.encode(answer_orig)
    v_orig_norm =  normalize(v_orig)
    evaluations_norm.append(v_llm_norm.dot(v_orig_norm))

Processing answers: 100%|██████████| 300/300 [01:23<00:00,  3.61it/s]


In [25]:
#Q3: What's the 75% cosine in the scores (norm)?

# Calculate the 75th percentile
percentile_75_norm = np.percentile(evaluations_norm, 75)

print(f"The 75th percentile of the scores is: {percentile_75_norm:.4f}")

The 75th percentile of the scores is: 0.8362


In [26]:
!pip install rouge


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [31]:
from rouge import Rouge
rouge_scorer = Rouge()


# try one

# Get the specific row from the dataframe
r = df.loc[df['document'] == '5170565b'].iloc[0]

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [32]:
scores


{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [33]:
# Q4: What's the F score for rouge-1?
scores['rouge-1']['f']

0.45454544954545456

In [34]:

# Extract F-scores for ROUGE-1, ROUGE-2, and ROUGE-L
f_score_1 = scores['rouge-1']['f']
f_score_2 = scores['rouge-2']['f']
f_score_l = scores['rouge-l']['f']

# Compute the average F-score
average_f_score = (f_score_1 + f_score_2 + f_score_l) / 3

print(f"F-scores:")
print(f"  ROUGE-1: {f_score_1:.4f}")
print(f"  ROUGE-2: {f_score_2:.4f}")
print(f"  ROUGE-L: {f_score_l:.4f}")
print(f"\nAverage F-score: {average_f_score:.4f}")

F-scores:
  ROUGE-1: 0.4545
  ROUGE-2: 0.2162
  ROUGE-L: 0.3939

Average F-score: 0.3549


In [35]:
# Q5 ans: Let's compute the average F-score between rouge-1, rouge-2 and rouge-l for the same record from Q4
print(average_f_score)


0.35490034990035496


In [37]:

# Prepare lists to store the scores
rouge_scores = []

# Compute ROUGE scores for each record
for _, row in tqdm(df.iterrows(), total=len(df), desc="Computing ROUGE scores"):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    
    rouge_scores.append({
        'document': row['document'],
        'rouge_1_f': scores['rouge-1']['f'],
        'rouge_2_f': scores['rouge-2']['f'],
        'rouge_l_f': scores['rouge-l']['f']
    })

# Create a new dataframe with the scores
rouge_df = pd.DataFrame(rouge_scores)

# Calculate the average ROUGE-2 score
average_rouge_2 = rouge_df['rouge_2_f'].mean()

print(f"The average ROUGE-2 F-score across all records is: {average_rouge_2:.4f}")

# Display the first few rows of the new dataframe
print("\nFirst few rows of the ROUGE scores dataframe:")
print(rouge_df.head())

Computing ROUGE scores: 100%|██████████| 300/300 [00:00<00:00, 339.32it/s]

The average ROUGE-2 F-score across all records is: 0.2070

First few rows of the ROUGE scores dataframe:
   document  rouge_1_f  rouge_2_f  rouge_l_f
0  0227b872   0.095238   0.028169   0.095238
1  0227b872   0.125000   0.055556   0.093750
2  0227b872   0.415584   0.177778   0.389610
3  0227b872   0.216216   0.047059   0.189189
4  0227b872   0.142076   0.033898   0.120219


In [38]:
# Q6: What's the average rouge_2 across all the records?

print(average_rouge_2)

0.20696501983423318
